In [3]:
pip install datasets

  Using cached datasets-2.15.0-py3-none-any.whl (521 kB)
  Using cached pyarrow-14.0.2-cp311-cp311-win_amd64.whl (24.6 MB)
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl (7.9 kB)
  Using cached dill-0.3.7-py3-none-any.whl (115 kB)
  Using cached xxhash-3.4.1-cp311-cp311-win_amd64.whl (29 kB)
  Using cached multiprocess-0.70.15-py311-none-any.whl (135 kB)
  Using cached fsspec-2023.10.0-py3-none-any.whl (166 kB)
  Using cached aiohttp-3.9.1-cp311-cp311-win_amd64.whl (364 kB)
  Using cached huggingface_hub-0.20.1-py3-none-any.whl (330 kB)
  Using cached multidict-6.0.4-cp311-cp311-win_amd64.whl (28 kB)
  Using cached yarl-1.9.4-cp311-cp311-win_amd64.whl (76 kB)
  Using cached frozenlist-1.4.1-cp311-cp311-win_amd64.whl (50 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached filelock-3.13.1-py3-none-any.whl (11 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: D:\Program Files\Python39\python.exe -m pip install --upgrade pip


In [1]:
from datasets import load_dataset

D:\Program Files\Python39\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

billsum = load_dataset("billsum", split="ca_test")

billsum = billsum.train_test_split(test_size=0.2)

In [3]:
billsum["train"][0]

{'text': 'The people of the State of California do enact as follows:\n\n\nSECTION 1.\nThe Legislature finds and declares all of the following:\n(a) Local educational agencies (LEA) must have an approved provider participant agreement with the State Department of Health Care Services through the federal Centers for Medicare and Medicaid Services to be eligible to provide services. To participate in the LEA Medi-Cal billing option program, LEAs must reinvest the federal reimbursement they receive under this program in health and social services for children and families, and develop and maintain a collaborative committee to assist them in decisions regarding the reinvestment of federal reimbursements. The providers and supervisors of staff for the assessment and medically necessary health services are those qualified medical practitioners the LEA employs or contracts with to render certain health services.\n(b) The LEA billing option facilitates reinvestment in health and social services

In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small")

tokenizer_config.json: 100%|██████████████████████████████████████████████████████████████| 2.32k/2.32k [00:00<?, ?B/s]
spiece.model: 100%|██████████████████████████████████████████████████████████████████| 792k/792k [00:00<00:00, 828kB/s]
tokenizer.json: 100%|█████████████████████████████████████████████████████████████| 1.39M/1.39M [00:00<00:00, 1.90MB/s]


In [14]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [15]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map: 100%|███████████████████████████████████████████████████████████████████| 248/248 [00:00<00:00, 281.40 examples/s]


In [16]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, return_tensors="tf")

NameError: name 'model' is not defined

In [4]:
from transformers import pipeline

In [5]:
summarizer = pipeline("summarization", model="Falconsai/text_summarization")

config.json: 100%|████████████████████████████████████████████████████████████████| 1.49k/1.49k [00:00<00:00, 1.50MB/s]
model.safetensors: 100%|████████████████████████████████████████████████████████████| 242M/242M [02:24<00:00, 1.68MB/s]
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
tokenizer_config.json: 100%|██████████████████████████████████████████████████████| 2.32k/2.32k [00:00<00:00, 2.33MB/s]
spiece.model: 100%|██████████████████████████████████████████████████████████████████| 792k/792k [00:00<00:00, 924kB/s]
tokenizer.json: 100%|█████████████████████████████████████████████████████████████| 2.42M/2.42M [00:01<00:00, 2.11MB/s]
special_tokens_map.json: 100%|█████████████████████████

In [6]:
ARTICLE = "La verdad ella no explica mucho, todo va hacia los estudiantes quiénes buscan la información para exponerla ese mismo día o a la siguiente clase. Ubica a todos los que entran así que si faltas no esperes que al final te pasé solo porque si. Algo aburrida y quita tiempo la materia pero 10 seguro."

In [7]:
print(summarizer(ARTICLE, max_length=230, min_length=30, do_sample=False))

Your max_length is set to 230, but your input_length is only 124. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=62)


[{'summary_text': 'Ubica a todos los estudiantes quiénes buscan la información para exponerla ese mismo da o a la siguiente clase. Algo aburrida y quita tiempo la materia pero 10 seguro.'}]


In [8]:
a2 = "Sabe enseñar, cubre bien los temas, es una persona muy linda y le gusta que participen. Deja de 1 a 3 tareas por semana, deja un proyecto por equipo que se expone al final del semestre, en línea no aplicó examen"

In [9]:
print(summarizer(a2, max_length=230, min_length=30, do_sample=False))

Your max_length is set to 230, but your input_length is only 90. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=45)


[{'summary_text': 'Deja de 1 a 3 tareas por semana, deja un proyecto por equipo que se expone al final del semestre, en lnea no aplicó examen .'}]


In [12]:
a3 = "Uno de los aspectos fundamentales de la IA es el aprendizaje automático, que permite a las máquinas mejorar su rendimiento a medida que se enfrentan a nuevas situaciones y datos. El aprendizaje profundo, una rama específica del aprendizaje automático, utiliza redes neuronales artificiales para simular el procesamiento de información del cerebro humano. Esto ha llevado a avances significativos en áreas como el reconocimiento de patrones, la visión por computadora y el procesamiento del lenguaje natural. La IA se implementa en una variedad de aplicaciones cotidianas, como motores de búsqueda en internet, asistentes virtuales, recomendaciones personalizadas en plataformas de transmisión de contenido y sistemas de reconocimiento facial. Estos avances tecnológicos han transformado la forma en que interactuamos con la tecnología y entre nosotros, creando un impacto profundo en la sociedad y la economía."

In [13]:
print(summarizer(a3, max_length=230, min_length=30, do_sample=False))

[{'summary_text': 'El aprendizaje profundo utiliza redes neuronales artificiales para simular el procesamiento de información del cerebro humano . La IA se implementa en una variedad de aplicaciones cotidianas, como motores de bsqueda, asistentes virtuales recomendaciones personalizadas en plataformas de transmisión de contenido y'}]


In [14]:
summarizer2 = pipeline("summarization", model="facebook/bart-large-cnn")


config.json: 100%|████████████████████████████████████████████████████████████████████████| 1.58k/1.58k [00:00<?, ?B/s]
D:\Program Files\Python39\Lib\site-packages\huggingface_hub\file_download.py:983: UserWarning: Not enough free disk space to download the file. The expected file size is: 1625.22 MB. The target location C:\Users\ricar\.cache\huggingface\hub only has 1473.69 MB free disk space.
  warnings.warn(
D:\Program Files\Python39\Lib\site-packages\huggingface_hub\file_download.py:983: UserWarning: Not enough free disk space to download the file. The expected file size is: 1625.22 MB. The target location C:\Users\ricar\.cache\huggingface\hub\models--facebook--bart-large-cnn\blobs only has 1473.69 MB free disk space.
  warnings.warn(
model.safetensors:  20%|███████████▊                                               | 325M/1.63G [02:07<08:28, 2.55MB/s]


ValueError: Could not load model facebook/bart-large-cnn with any of the following classes: (<class 'transformers.models.auto.modeling_tf_auto.TFAutoModelForSeq2SeqLM'>, <class 'transformers.models.bart.modeling_tf_bart.TFBartForConditionalGeneration'>). See the original errors:

while loading with TFAutoModelForSeq2SeqLM, an error is thrown:
Traceback (most recent call last):
  File "D:\Program Files\Python39\Lib\site-packages\transformers\pipelines\base.py", line 269, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Program Files\Python39\Lib\site-packages\transformers\models\auto\auto_factory.py", line 566, in from_pretrained
    return model_class.from_pretrained(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Program Files\Python39\Lib\site-packages\transformers\modeling_tf_utils.py", line 2785, in from_pretrained
    resolved_archive_file = cached_file(pretrained_model_na